In [ ]:
import json
import os
import sys
import math
import argparse
import numpy as np
from scipy.special import erf
from scipy.interpolate import interp1d
from mpl_toolkits.mplot3d import Axes3D
from shutil import rmtree, copy
from matplotlib import pyplot as plt
from matplotlib import gridspec, cm
from matplotlib import patches as mpatches 

plt.style.use('publication_lhy11009')

R = 8.314

# directory to the aspect Lab
ASPECT_LAB_DIR = os.environ['ASPECT_LAB_DIR']
RESULT_DIR = os.path.join(ASPECT_LAB_DIR, 'results')

sys.path.append(os.path.join(ASPECT_LAB_DIR))
import shilofue.PlotDepthAverage as PDAver
import shilofue.FlowLaws as FlowLaws
import shilofue.ParsePrm as ParsePrm
import shilofue.ThermalModel as TModel
import shilofue.Rheology as Rheology

# import utilities in subdirectiory
sys.path.append(os.path.join(ASPECT_LAB_DIR, 'utilities', "python_scripts"))
import Utilities


In [ ]:
# read the parameters from HK 03.
# water is in the water fugacity
rheology = "HK03"
use_coh = True

rheology_prm_dict = Rheology.RHEOLOGY_PRM()
diffusion_creep = getattr(rheology_prm_dict, rheology + "_diff")
dislocation_creep = getattr(rheology_prm_dict, rheology + "_disl")
rheology_dict = {'diffusion': diffusion_creep, 'dislocation': dislocation_creep}

print("diffusion_creep: ", diffusion_creep)
print("dislocation_creep: ", dislocation_creep)

In [ ]:
creep = diffusion_creep
creep_correction = {'A': 1.0, 'p': 0.0, 'r': 0.0, 'n': 0.0, 'E': 75e3, 'V': 5e-06}

n = creep['n']
F = 1 / (2**((n-1)/n)*3**((n+1)/2/n))


# fit new rheology
stress_ref = 50.0 # MPa
# strain_rate_ref = 1e-5
P_ref = 100.0e6 # Pa
T_ref = 1250.0 + 273.15 # K
Coh_ref = 1000.0 # H / 10^6 Si
d_ref = 15.0 # mu m

strain_rate_correction = Rheology.CreepStrainRate(creep_correction, stress_ref, P_ref, T_ref, d_ref, Coh_ref, f_by_factor=True, F=F)
# eta_correction = Rheology.CreepRheology(creep_correction, strain_rate_ref, P_ref, T_ref, d=d_ref, Coh=Coh_ref, f_by_factor=True, F=F)